# Application of HMLasso

The main goal of this notebook is to use the previously implemented HMLasso to critically reduce the volume of columns.

## Imports

In [ ]:
!cp "/content/drive/MyDrive/Statapp/file_04_HMLasso.py" "HMLasso.py"

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler # To standardize the data

# from HMLasso import HMLasso # Lasso with High Missing Rate

In [ ]:
columns_types = pd.read_csv("/content/drive/MyDrive/Statapp/data_03_columns_types.csv")
data = pd.read_csv("/content/drive/MyDrive/Statapp/data_03.csv")

<ipython-input-2-1d21f6d3a7c2>:2: DtypeWarning: Columns (3970) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/Statapp/data_03.csv")


In [ ]:
data_GHI = pd.read_csv("/content/drive/MyDrive/Statapp/data_only_health_index_2.csv")

## Formatting the database

In [ ]:
data_GHI.head()

,HHIDPN,index_2_w1,index_2_w2,index_2_w3,index_2_w4,index_2_w5,index_2_w6,index_2_w7,index_2_w8,index_2_w9,index_2_w10,index_2_w11,index_2_w12,index_2_w13,index_2_w14
0,1010,0.333088,-3.698928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,0.246520,0.255046,0.213209,0.307378,0.312587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010,0.363632,0.431329,0.335114,0.320731,0.323663,0.330785,0.364091,0.318365,0.320656,0.257284,0.158740,NaN,NaN,NaN
3,3020,0.437208,0.334530,0.390284,0.420346,0.378833,0.235120,0.149134,0.304341,0.309083,0.276126,0.142465,-0.066251,NaN,NaN
4,10001010,0.411677,0.299593,0.321996,0.270861,0.300019,0.294110,0.307931,0.287869,0.269260,0.308847,0.227056,0.275555,0.271741,NaN


In [ ]:
# We change the names just because I want it.
new_columns_names = ["HHIDPN"] + [f"GHI{col[9:]}" for col in data_GHI.columns if "index" in col]
data_GHI.columns = new_columns_names
data_GHI.head()

,HHIDPN,GHI1,GHI2,GHI3,GHI4,GHI5,GHI6,GHI7,GHI8,GHI9,GHI10,GHI11,GHI12,GHI13,GHI14
0,1010,0.333088,-3.698928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,0.246520,0.255046,0.213209,0.307378,0.312587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010,0.363632,0.431329,0.335114,0.320731,0.323663,0.330785,0.364091,0.318365,0.320656,0.257284,0.158740,NaN,NaN,NaN
3,3020,0.437208,0.334530,0.390284,0.420346,0.378833,0.235120,0.149134,0.304341,0.309083,0.276126,0.142465,-0.066251,NaN,NaN
4,10001010,0.411677,0.299593,0.321996,0.270861,0.300019,0.294110,0.307931,0.287869,0.269260,0.308847,0.227056,0.275555,0.271741,NaN


In [ ]:
for hhidpn in data["HHIDPN"].values:
  if hhidpn not in data_GHI["HHIDPN"].values:
    print(hhidpn)

32570030


In [ ]:
for hhidpn in data_GHI["HHIDPN"].values:
  if hhidpn not in data["HHIDPN"].values:
    print(hhidpn)

In [ ]:
# We see that one individual is missing in our data. Let us drop it.
data = data[data["HHIDPN"] != 32570030]

In [ ]:
data = data.merge(data_GHI, on="HHIDPN")
data.head()

,HHIDPN,R1MPART,R2MPART,R3MPART,R4MPART,R5MPART,R6MPART,R7MPART,R8MPART,R9MPART,...,GHI5,GHI6,GHI7,GHI8,GHI9,GHI10,GHI11,GHI12,GHI13,GHI14
0,1010,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0.312587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.323663,0.330785,0.364091,0.318365,0.320656,0.257284,0.158740,NaN,NaN,NaN
3,3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.378833,0.235120,0.149134,0.304341,0.309083,0.276126,0.142465,-0.066251,NaN,NaN
4,10001010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300019,0.294110,0.307931,0.287869,0.269260,0.308847,0.227056,0.275555,0.271741,NaN


Let us start by spotting temporal variables.

We will use the code from a precedent notebook to do so.

In [ ]:
temporal_variables = {}
waves_columns = [col for col in data.columns if "genetic_" not in col and col[1] in "123456789"]
for col in waves_columns:
  char = col[0] # R or H
  if col[2] in "01234":
    wave = col[1:3]
    suffix = col[3:]
  else:
    wave = col[1]
    suffix = col[2:]
  variable = char + 'w' + suffix
  
  if variable not in temporal_variables.keys():
    temporal_variables[variable] = np.zeros((14), dtype=bool)
  
  temporal_variables[variable][int(wave)-1] = True

temporal_variables = pd.DataFrame(temporal_variables)

# We manually add "GHIw":
temporal_variables["GHIw"] = np.ones((14), dtype=bool)
waves_columns += [f"GHI{w}" for w in range(1,15)]

In [ ]:
temporal_variables

,RwMPART,RwMLEN,RwMCURLN,RwMLENM,RwMNEV,HwANYFIN,HwANYFAM,RwFAMR,RwFINR,HwHHRESP,...,RwADL6A_4.0,RwADL6A_5.0,RwADL6A_6.0,RwIADL5H_0,RwIADL5H_1,RwIADL5H_2,RwIADL5H_3,RwIADL5H_4,RwIADL5H_5,GHIw
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,False,False,False,False,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


## Don't do this at home !

In this section, we will experiment what happen if we wanted to simply line up all variables from all waves, regardless of whether the variable is present for this wave.

In [ ]:
columns_wave1 = [col.replace('w', str(1)) for col in temporal_variables.T[0].index[temporal_variables.T[0]]]
non_waves_columns = [col for col in data.columns if col not in waves_columns]

data_wave1 = data.loc[data["INW1"] == 1, columns_wave1 + non_waves_columns]
data_wave1.head()


,R1MPART,R1MLEN,R1MCURLN,R1MLENM,R1MNEV,H1ANYFIN,H1ANYFAM,R1FAMR,R1FINR,H1HHRESP,...,REXITWV_14.0,REPLDEATH_1.0,REPLDEATH_2.0,REPLDEATH_3.0,REPLDEATH_4.0,REPLDEATH_5.0,REPLDEATH_7.0,REXPDEATH_1.0,REXPDEATH_2.0,REXPDEATH_7.0
0,0.0,20.2,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,25.8,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,31.3,31.3,0.0,0.0,1.0,1.0,0.0,1.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,31.2,31.2,0.0,0.0,1.0,1.0,1.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns_to_drop_in_Xy = ["HHIDPN", "HHID", "PN"] + [f"INW{w}" for w in range(1,15)] + ["genetic_Section_A_or_E"] + ["GHI1"]
X = data_wave1.drop(columns=columns_to_drop_in_Xy).values
y = data_wave1["GHI1"].values

In [ ]:
print(X.shape, y.shape)

(12651, 1128) (12651,)


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
lasso = HMLasso(mu = 100, verbose = 2)
lasso.fit(X, y)

[Imputing parameters] Starting...
[Imputing parameters] R calculated.
[Imputing parameters] rho_pair calculated.
[Imputing parameters] S_pair calculated.
[Imputing parameters] Parameters imputed.
[First Problem] Starting...
[First Problem] Objective and constraints well-defined.
[First Problem] Problem status: optimal.
[First Problem] Problem solved.
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is (-3.1363769993782416e-05+0j). Error handled by adding (3.1363769993782416e-05-0j) to each eigenvalue.
[Second Problem] Starting...
[Second Problem] Objective and constraints well-defined.


ArpackNoConvergence: ignored

In [ ]:
np.linalg.eig(lasso.Sigma_opt)[0]

array([4.41469352e+01+0.00000000e+00j, 3.35591279e+01+0.00000000e+00j,
       1.97289402e+01+0.00000000e+00j, ...,
       3.13638340e-05-3.04934315e-23j, 3.13638339e-05-5.80804746e-25j,
       3.13638340e-05+8.73582601e-24j])

Everything works well except the 'lasso.fit(X, y)':

The First problem is solved.
The Second problem fails to be solved due to the same error as [there](https://stackoverflow.com/questions/63117123/cvxpy-quadratic-programming-arpacknoconvergence-error).


[I also noticed that lasso.Sigma_opt has been casted as a complex matrix. Maybe applying "self.Sigma_opt = np.real(self.Sigma_opt)" might be an option. 
EDIT: Non, ça change rien mdr.]

Voici l'erreur :
ArpackNoConvergence                       Traceback (most recent call last)
/usr/local/lib/python3.9/dist-packages/cvxpy/utilities/linalg.py in is_psd_within_tol(A, tol)
     79     try:
---> 80         ev = SA_eigsh(-tol)  # might return np.NaN, or raise exception
     81     except sparla.ArpackNoConvergence as e:

31 frames
ArpackNoConvergence: ARPACK error -1: No convergence (11281 iterations, 0/1 eigenvectors converged) [ARPACK error -14: ZNAUPD did not find any eigenvalues to sufficient accuracy.]

During handling of the above exception, another exception occurred:

ArpackNoConvergence                       Traceback (most recent call last)
/usr/local/lib/python3.9/dist-packages/cvxpy/utilities/linalg.py in is_psd_within_tol(A, tol)
     94         error_with_note = f"{str(e)}\n\n{message}"
     95 
---> 96         raise sparla.ArpackNoConvergence(error_with_note, e.eigenvalues, e.eigenvectors)
     97 
     98     if np.isnan(ev).any():

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (11281 iterations, 0/1 eigenvectors converged) [ARPACK error -14: ZNAUPD did not find any eigenvalues to sufficient accuracy.]


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix

In [ ]:
# Imports
import numpy as np
import pandas as pd

import cvxpy as cp
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

class HMLasso():
  """
  Lasso regularization that performs well with high missing rate.

  Implemented according to the related article 'HMLasso: Lasso with High Missing
  Rate' by Masaaki Takada1, Hironori Fujisawa and Takeichiro Nishikawa.
  Link to the article: https://www.ijcai.org/proceedings/2019/0491.pdf

  ------------
  Common uses: Once fitted, the HMLasso can provide linear predictions. 
  It can also be used to select variables of interest from the given data. This 
  second goal can be achieved through selection of variables whose coefficient
  is almost (or equal to) zero.

  Please note that no metric is implemented in this class for now. 
  See sklearn.metrics.mean_squared_error or like for useful metrics.

  ------------
  Parameters:
      mu : float/int, default=1.0: the hyperparameter that control how
      parcimonious the model shall be. The larger mu is, the greater the
      regularization will be (hence the calculated beta_opt might 
      present more nullified coefficients). mu must be positive.
      
      alpha : float/int, default=1: the hyperparameter that control weights
      importance. Be wary that setting alpha > 5 can make convergence way
      slower, as the weights become closer and closer to 0 and as the numerical
      solver has more and more trouble converging.
      One may prefer setting alpha in the range [0., 3.]. Common values
      of alpha are 0., 0.5, 1. with the latter experimentally delivering best
      performances. alpha must be positive.
      See source article for more.

      verbose : float/int, default=1: control how much verbose
      is displayed. Encoded values are 0, 1 and 2. If verbose > 2, there
      will be no difference with verbose=2 display.
  
  ------------
  Methods:
      fit(self, X, y):
        Fit the HMLasso on (X, y)
        X, the features, must be a mean-centered numpy array of shape (n, p)
        y, the labels, must be a vector of shape (n, 1) or (n,)

        Do not return anything. However, once the fitting is done, one can
        use 'predict' method to predict any given output using the linear model.
      
      predict(self, X):
        Predict using linear model.
        Return the predicted vector.
  
  ------------
  Constants:
      beta_opt: the estimator.


  """

  def __init__(self, mu=1, alpha=1, verbose=1):

    assert type(mu) is int or type(mu) is float, "mu must be a number."
    assert type(alpha) is int or type(alpha) is float, "alpha must be a number."
    assert type(verbose) is int, "verbose must be an integer."
    assert mu >= 0, "mu must be a positive number."
    assert alpha >= 0, "alpha must be a positive number."

    self.mu = mu
    self.alpha = alpha
    self.verbose = verbose
    
    self.n = None
    self.p = None
    self.S_pair = None
    self.rho_pair = None
    self.R = None
    self.Sigma_opt = None
    self.beta_opt = None

    self.isFirstProblemSolved = False
    self.isSecondProblemSolved = False # Unused at the moment.
    self.isFitted = False
  
  def predict(self, X):
    """
    Predict using the linear model.

    ------------
    Parameters:
        X : 2D numpy array

    Returns:
        y : 1D numpy array
    """

    assert self.isFitted, "The model has not yet been fitted."
    assert X.shape[1] == self.p, f"Given data is of dimension {X.shape[1]}. Must have dimension {self.p})."
    assert not np.isnan(X).any(), "Input contains NaN."

    return np.dot(X, self.beta_opt)
  
  def fit(self, X, y):
    """
    Fit the HMLasso on (X, y).

    ------------
    Parameters:
        X : 2D numpy array, shape (n,p). It corresponds to the features, and
        must be mean-centered.
        y : 1D numpy array, shape (n,1) or (n,). It corresponds to the labels.

    Returns:
        None
    """
    
    assert type(X) == np.ndarray, "Features are not a numpy array."
    assert type(y) == np.ndarray, "Labels are not a numpy array"
    assert X.shape[0] == y.shape[0], "Features and labels shapes are not compatibles."
    assert len(y.shape) == 1, "Labels are not a vector."

    self.n, self.p = X.shape    
    self.__verify_centering__(X)
    self.S_pair, self.rho_pair, self.R = self.__impute_params__(X, y)
    self.Sigma_opt = self.__solve_first_problem__()

    # It appears that, due to floating points exceptions, Sigma_opt is not always
    # Real and Positive semidefinite. Hence, we shall check it.
    if np.iscomplex(self.Sigma_opt).any():
      raise Exception("Sigma_opt has complex values.")
    else:
      self.Sigma_opt = np.real(self.Sigma_opt)
    eigenvalues = np.linalg.eig(self.Sigma_opt)[0]
    min_eigenvalue = min(eigenvalues)
    if min_eigenvalue < 0:
      print(f"[Warning] Sigma_opt is not PSD, its minimum eigenvalue is {min_eigenvalue}. Error handled by adding {-min_eigenvalue} to each eigenvalue.")
      self.Sigma_opt = self.Sigma_opt - min_eigenvalue * np.eye(self.p, self.p)
    
    self.beta_opt = self.__solve_second_problem__()

    self.isFitted = True

    if self.verbose > 0:
      print("Model fitted.")

  def __verify_centering__(self, X, tolerance=1e-8):
    for col in range(self.p):
      current_mean = X[:, col].mean()
      if abs(current_mean) > tolerance:
        raise Exception(f"Data is not centered: column {col} has mean of {current_mean}")
  
  def __impute_params__(self, X, y):

    if self.verbose > 0:
      print("[Imputing parameters] Starting...")

    Z = np.nan_to_num(X)
    Y = (Z != 0).astype(int)
    R = np.dot(Y.T, Y)
    if self.verbose > 1:
      print("[Imputing parameters] R calculated.")

    rho_pair = np.divide(np.dot(Z.T, y), R.diagonal(), out=np.zeros((self.p,)), where=(R.diagonal()!=0))
    if self.verbose > 1:
      print("[Imputing parameters] rho_pair calculated.")

    S_pair = np.divide(np.dot(Z.T, Z), R, out=np.zeros((self.p, self.p)), where=(R!=0))
    if self.verbose > 1:
      print("[Imputing parameters] S_pair calculated.")

    R = R / self.n

    if self.alpha > 5:
      print("[Warning] The hyperparameter alpha={} is large (greater than 5), which might make convergence way slower.")
    R = np.power(R, self.alpha)

    if self.verbose > 0:
      print("[Imputing parameters] Parameters imputed.")

    return S_pair, rho_pair, R


  def __solve_first_problem__(self):
    
    assert self.S_pair is not None, "Pairwise covariance matrix of features is not determined."
    assert self.rho_pair is not None, "Pairwise covariance vector of features and labels is not determined."
    assert self.R is not None, "Weights are not determined."

    if self.verbose > 0:
      print("[First Problem] Starting...")

    Sigma = cp.Variable((self.p, self.p), PSD = True) # Variable to optimize
    obj = cp.Minimize(cp.sum_squares(cp.multiply(self.R, Sigma-self.S_pair))) # Objective to minimize
    constraints = [Sigma >> 0] # Constraints: We want Sigma to be positive semi-definite.
    if self.verbose > 1:
      print("[First Problem] Objective and constraints well-defined.")

    # Solve the optimization problem
    prob = cp.Problem(obj, constraints)
    prob.solve()
    if self.verbose > 1:
      print(f"[First Problem] Problem status: {prob.status}.")
    if self.verbose > 0:
      print("[First Problem] Problem solved.")

    self.isFirstProblemSolved = True

    return Sigma.value

  def __solve_second_problem__(self):
    
    assert self.S_pair is not None, "Pairwise covariance matrix of features is not determined."
    assert self.rho_pair is not None, "Pairwise covariance vector of features and labels is not determined."
    assert self.R is not None, "Weights are not determined."
    assert self.isFirstProblemSolved, " First optimization problem has not been solved."
    assert self.Sigma_opt is not None, "Sigma_opt is unknown. First optimization problem might have not been solved."

    if self.verbose > 0:
      print("[Second Problem] Starting...")

    beta = cp.Variable(self.p) # Variable to optimize
    obj = cp.Minimize(0.5 * cp.quad_form(beta, self.Sigma_opt) - self.rho_pair.T @ beta + self.mu * cp.norm1(beta)) # Objective to minimize
    constraints = [] # Constraints
    if self.verbose > 1:
      print("[Second Problem] Objective and constraints well-defined.")

    # Solve the optimization problem
    prob = cp.Problem(obj, constraints)
    prob.solve()
    if self.verbose > 1:
      print(f"[Second Problem] Problem status: {prob.status}.")
    if self.verbose > 0:
      print("[Second Problem] Problem solved.\n")
    
    self.isSecondProblemSolved = True

    return beta.value
